In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import timeit

from trackml.dataset import load_event, load_dataset
from trackml.score import score_event

from multiprocessing import Pool

from sklearn.neighbors import radius_neighbors_graph, kneighbors_graph
from sklearn.preprocessing import StandardScaler
import hdbscan
from scipy import stats
from tqdm import tqdm_notebook as tqdm
from sklearn.cluster import DBSCAN, AgglomerativeClustering

def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits.hit_id.values, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission

In [2]:
# Change this according to your directory preferred setting
path_to_train = "./data/train_1"

# This event is in Train_1
event_prefix = "event000001000"

In [3]:
class Clusterer(object):
    def __init__(self,rz_scales=[0.65, 0.965, 1.528], eps=0.0035):                        
        self.rz_scales=rz_scales
        self.epsilon = eps
        
    def _eliminate_outliers(self,labels,M):
        norms=np.zeros((len(labels)),np.float32)
        indices=np.zeros((len(labels)),np.float32)
        for i, cluster in tqdm(enumerate(labels),total=len(labels)):
            if cluster == 0:
                continue
            index = np.argwhere(self.clusters==cluster)
            index = np.reshape(index,(index.shape[0]))
            indices[i] = len(index)
            x = M[index]
            norms[i] = self._test_quadric(x)
        threshold1 = np.percentile(norms,90)*5
        threshold2 = 25
        threshold3 = 6
        for i, cluster in enumerate(labels):
            if norms[i] > threshold1 or indices[i] > threshold2 or indices[i] < threshold3:
                self.clusters[self.clusters==cluster]=0   
    
    def _test_quadric(self,x):
        if x.size == 0 or len(x.shape)<2:
            return 0
        Z = np.zeros((x.shape[0],10), np.float32)
        Z[:,0] = x[:,0]**2
        Z[:,1] = 2*x[:,0]*x[:,1]
        Z[:,2] = 2*x[:,0]*x[:,2]
        Z[:,3] = 2*x[:,0]
        Z[:,4] = x[:,1]**2
        Z[:,5] = 2*x[:,1]*x[:,2]
        Z[:,6] = 2*x[:,1]
        Z[:,7] = x[:,2]**2
        Z[:,8] = 2*x[:,2]
        Z[:,9] = 1
        v, s, t = np.linalg.svd(Z,full_matrices=False)        
        smallest_index = np.argmin(np.array(s))
        T = np.array(t)
        T = T[smallest_index,:]        
        norm = np.linalg.norm(np.dot(Z,T), ord=2)**2
        return norm

    def _preprocess(self, hits):
        
        x = hits.x.values
        y = hits.y.values
        z = hits.z.values

        r = np.sqrt(x**2 + y**2 + z**2)
        hits['x2'] = x/r
        hits['y2'] = y/r

        r = np.sqrt(x**2 + y**2)
        hits['z2'] = z/r

        ss = StandardScaler()
        X = ss.fit_transform(hits[['x2', 'y2', 'z2']].values)
        for i, rz_scale in enumerate(self.rz_scales):
            X[:,i] = X[:,i] * rz_scale
       
        return X
    
    def _init(self,dfh):
        dfh['r'] = np.sqrt(dfh['x'].values**2+dfh['y'].values**2+dfh['z'].values**2)
        dfh['rt'] = np.sqrt(dfh['x'].values**2+dfh['y'].values**2)
        dfh['a0'] = np.arctan2(dfh['y'].values,dfh['x'].values)
        dfh['z1'] = dfh['z'].values/dfh['rt'].values
        dfh['x2'] = 1/dfh['z1'].values
        dz0 = -0.00070
        stepdz = 0.00001
        stepeps = 0.000005
        mm = 1
        for ii in tqdm(range(100)):
            mm = mm*(-1)
            dz = mm*(dz0+ii*stepdz)
            dfh['a1'] = dfh['a0'].values+dz*dfh['z'].values*np.sign(dfh['z'].values)
            dfh['sina1'] = np.sin(dfh['a1'].values)
            dfh['cosa1'] = np.cos(dfh['a1'].values)
            dfh['x1'] = dfh['a1'].values/dfh['z1'].values
            ss = StandardScaler()
            dfs = ss.fit_transform(dfh[['sina1','cosa1','z1','x1','x2']].values)
            cx = np.array([1, 1, 0.75, 0.5, 0.5])
            dfs = np.multiply(dfs, cx)

            clusters=DBSCAN(eps=self.epsilon+ii*stepeps,min_samples=1,metric='euclidean', n_jobs=8).fit(dfs).labels_            
            
            if ii==0:
                dfh['s1'] = clusters
                dfh['N1'] = dfh.groupby('s1')['s1'].transform('count')
            else:
                dfh['s2'] = clusters
                dfh['N2'] = dfh.groupby('s2')['s2'].transform('count')
                maxs1 = dfh['s1'].max()
                cond = np.where((dfh['N2'].values>dfh['N1'].values) & (dfh['N2'].values<20))
                s1 = dfh['s1'].values
                s1[cond] = dfh['s2'].values[cond]+maxs1
                dfh['s1'] = s1
                dfh['s1'] = dfh['s1'].astype('int64')
                dfh['N1'] = dfh.groupby('s1')['s1'].transform('count')
                
        return dfh['s1'].values    
    
    def predict(self, hits):    
        self.clusters = self._init(hits) 
        X = self._preprocess(hits) 
        
#         cl = hdbscan.HDBSCAN(min_samples=1,min_cluster_size=7,
#                              metric='braycurtis',cluster_selection_method='leaf',algorithm='best', leaf_size=50)
    
#         connectivity = radius_neighbors_graph(X, 200, mode="connectivity", include_self=True)
    
#         connectivity = kneighbors_graph(X[mask], 200, mode="connectivity", include_self=True, n_jobs=8)
    
        labels = np.unique(self.clusters)
        self._eliminate_outliers(labels,X)
        n_labels = 0
        while n_labels < len(labels):
            n_labels = len(labels)            
            max_len = np.max(self.clusters)
            mask = self.clusters == 0
            connectivity = kneighbors_graph(X[mask], 200, mode="connectivity", include_self=True, n_jobs=8)
#             connectivity = radius_neighbors_graph(X[mask], 0.15, mode="connectivity", include_self=True, n_jobs=8)
            cl = AgglomerativeClustering(n_clusters=len(np.unique(self.clusters)), connectivity=connectivity, affinity="cosine", linkage="complete")
            self.clusters[mask] = cl.fit_predict(X[mask])+max_len
        return self.clusters

In [4]:
path_to_test = "./data/test"
eps = 0.0035

def one_loop(event_id):
    hits  = pd.read_csv(path_to_test + '/event%s-hits.csv'%event_id)
    cells = pd.read_csv(path_to_test + '/event%s-cells.csv'%event_id)
    print('Event ID: ', event_id)
                
    # Track pattern recognition 
    model = Clusterer(eps=eps)
    labels = model.predict(hits)

    # Prepare submission for an event
    one_submission = create_one_event_submission(event_id, hits, labels)
    one_submission.to_csv('./%09d_knn_agg_dbscan_e_35'%int(event_id), index=False, compression='gzip')
            
    return one_submission

def create_test_submissions(path_to_test = "./data/test", start=0, end=125):
    event_ids = [ '%09d'%i for i in range(start,end) ]

    pool = Pool(processes=2)
    results = pool.map(one_loop, event_ids)
    pool.close()
    
    return results

In [ ]:
submission = create_test_submissions(start=55, end=125)

Event ID:  000000055
Event ID:  000000064



Event ID:  000000056


Event ID:  000000065



Event ID:  000000057


Event ID:  000000066


...

Event ID:  000000102



Event ID:  000000095


Event ID:  000000103



Event ID:  000000096


Event ID:  000000104


In [4]:
event_ids = [ i for i in range(0,125) ]
submissions = []
for i,event_id in enumerate(event_ids):
    submission  = pd.read_csv('./%09d.agg.csv.gz'%event_id, compression='gzip')
    submissions.append(submission)

# Create submission file
submission = pd.concat(submissions, axis=0)
submission.to_csv('20180703_knn_agg_dbscan_e_37.csv.gz', index=False, compression='gzip')
print(len(submission))

13741466


In [4]:
## Run on test data, sequentially
start = 1
dataset_submissions = []
dataset_scores = []
for event_id, hits, cells in load_dataset(path_to_test, parts=['hits', 'cells']):
    if event_id >= start:
        print('Event ID: ', event_id)

        # Track pattern recognition 
        model = Clusterer(eps=0.0037)
        labels = model.predict(hits)

        # Prepare submission for an event
        one_submission = create_one_event_submission(event_id, hits, labels)
        one_submission.to_csv('./%09d.agg.csv.gz'%event_id, index=False, compression='gzip')

NameError: name 'path_to_test' is not defined

In [4]:
dataset_submissions = []
dataset_scores = []
for event_id, hits, cells, particles, truth in load_dataset(path_to_train, skip=0, nevents=5):
    # Track pattern recognition
    model = Clusterer(eps=0.0037)
    labels = model.predict(hits)

    # Prepare submission for an event
    one_submission = create_one_event_submission(event_id, hits, labels)
    dataset_submissions.append(one_submission)

    # Score for the event
    score = score_event(truth, one_submission)
    dataset_scores.append(score)

    print("Score for event %d: %.8f" % (event_id, score))
print('Mean score: %.8f' % (np.mean(dataset_scores)))

TypeError: 'float' object cannot be interpreted as an integer

In [6]:
print("X:", X.shape)

X: (120939, 3)


In [ ]:
foo = kneighbors_graph(X, 100, mode="connectivity", include_self=True)
# foo = radius_neighbors_graph(X, 50, mode="connectivity", include_self=True)
print(foo.shape)

In [8]:
foo

<120939x120939 sparse matrix of type '<class 'numpy.float64'>'
	with 846573 stored elements in Compressed Sparse Row format>

In [ ]:
# predict one hit
model = Clusterer(eps=0.0040)
labels = model.predict(hits)

submission = create_one_event_submission(0, hits, labels)
score = score_event(truth, submission)
print("Your score: ", score)